# Parameter-Efficient Fine-Tuning (PEFT) with NeMo

In this example, we utilize NeMo's [PEFT](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/nemo_megatron/peft/landing_page.html)
methods to showcase how to adapt a large language model (LLM) to 
a downstream task, such as financial sentiment predictions. 

With one line configuration change, you can try different PEFT techniques such as [p-tuning](https://arxiv.org/abs/2103.10385), [adapters](https://proceedings.mlr.press/v97/houlsby19a.html), or [LoRA](https://arxiv.org/abs/2106.09685), which add a small number of trainable parameters to the LLM
that condition the model to produce the desired output for the downstream task.

For more details, see the [PEFT script](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/megatron_gpt_peft_tuning.py) in NeMo, which we adapt using NVFlare's Lightning client API to run in a federated scenario.

## Dependencies
We assume you followed the instructions [here](./README.md) 
to install the NeMo and NVFlare frameworks and mount the required codes.

## Download the pre-trained LLM
In this example, we use a `MegatronGPTModel`, a transformer-based language model based on the GPT architecture.

In [1]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2025-06-09 20:02:32 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

In [2]:
# Download the model from NGC
import os
model_file = "megatron_gpt_345m.nemo"
if not os.path.isfile(model_file):
    !wget "https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/$model_file"
else:
    print(f"{model_file} already downloaded.")

megatron_gpt_345m.nemo already downloaded.


## Data preprocessing
As our downstream task, we will use the [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for sentiment analysis.

The Financial PhraseBank dataset contains the sentiments for financial news headlines from a retail investor's perspective. Further details about the dataset can be found in Malo et al.'s ["Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts"](https://arxiv.org/abs/1307.5336).

We can configure the prompt template used by NeMo to solve this downstream task by setting `prompt_template: "{sentence} sentiment: {label}"` in [megatron_gpt_peft_tuning_config.yaml](./nemo_nvflare/megatron_gpt_peft_tuning_config.yaml) accordingly.

#### 1. Download the preprocessing scripts
We use the preprocessing scripts provided by NeMo which can be downloaded from GitHub.

In [3]:
script_name = "prompt_learning_financial_phrase_bank_preprocessing.py"
if not os.path.isfile(script_name):
    !wget -N "https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/nlp/financial_phrase_bank/$script_name"
else:
    print(f"{script_name} already downloaded.")

prompt_learning_financial_phrase_bank_preprocessing.py already downloaded.


#### 2. Download the Financial PhraseBank Dataset

Download the `FinancialPhraseBank-v1.0.zip` dataset from [here](https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v1.0/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v1.0.zip).

Then extract it under `./data`.

#### 3. Preprocess the dataset

In [4]:
!python3 prompt_learning_financial_phrase_bank_preprocessing.py

Saving train split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl
100%|███████████████████████████████████| 1811/1811 [00:00<00:00, 220631.01it/s]
Saving val split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl
100%|█████████████████████████████████████| 226/226 [00:00<00:00, 250704.23it/s]
Saving test split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_test.jsonl
100%|█████████████████████████████████████| 227/227 [00:00<00:00, 278719.85it/s]


#### 4. Split the dataset to simulate clients
Next, we use three clients to simulate federated learning for running PEFT with NeMo. 
We use a [Dirichlet sampling](https://arxiv.org/abs/2002.06440) strategy for creating a heterogeneous partition. Smaller values of `alpha` cause higher heterogeneity.

In [5]:
from data.split_financial_phrase_data import clean_memmap

# Clean NeMo memmap data before running a new data split
clean_memmap("./data")

# Split the data
alpha = 10.0
assert isinstance(alpha, float), "Expecting float value in filepath names used below."
!python3 data/split_financial_phrase_data.py --alpha={alpha} --data_path=data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl --num_clients=3 --out_dir=data/FinancialPhraseBank-v1.0_split

Loaded training data with 1811 entries
After split Dirichlet sampling with alpha=10.0
{'site-1': {' negative': 75, ' neutral': 232, ' positive': 129},
 'site-2': {' negative': 76, ' neutral': 421, ' positive': 205},
 'site-3': {' negative': 95, ' neutral': 454, ' positive': 124}}
Save split 1 of 3 with 436 entries to data/FinancialPhraseBank-v1.0_split/alpha10.0_site-1.jsonl
Save split 2 of 3 with 702 entries to data/FinancialPhraseBank-v1.0_split/alpha10.0_site-2.jsonl
Save split 3 of 3 with 673 entries to data/FinancialPhraseBank-v1.0_split/alpha10.0_site-3.jsonl


Below are some examples of how the training data is distributed amount the three clients when using different values of `alpha`.
<div>
<img src="./figs/summary_alpha1.0.svg" alt="Label distribution with alpha=1.0" style="width: 400px;"/>
<img src="./figs/summary_alpha5.0.svg" alt="Label distribution with alpha=5.0" style="width: 400px;"/>
<img src="./figs/summary_alpha10.0.svg" alt="Label distribution with alpha=10.0" style="width: 400px;"/>
</div>

## Federated learning simulations
Next, we are using NVFlare's [simulator](https://nvflare.readthedocs.io/en/latest/user_guide/nvflare_cli/fl_simulator.html) to simulate each client training on their own dataset locally and all three clients training together using the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm implemented in NVFlare.

With this setting, we require a GPU with at least 24GB of memory to run all clients in parallel on the same GPU. 
If you have multiple GPUs in your system, you can use the `gpu` argument to assign one GPU for each client, e.g., `gpu="0,1"`.

We will use NVFlare's job command for each setting to create the configurations needed to train the models based on the [sag_nemo](https://github.com/NVIDIA/NVFlare/blob/main/job_templates/sag_nemo/info.md) job template. This template allows the definition of different configurations for each client, which we will use to assign their local training data file to each of them.

#### 1. Convert NeMo PEFT script to FL

To run NeMo in an FL scenario, we convert the NeMo [PEFT script](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/megatron_gpt_peft_tuning.py) using the new lightning client API. 

This conversion can be done with only a few lines of code changes, as highlighted in the figure below:

1. Import nvflare lightning api
2. Patch your lightning trainer
3. (Optionally) validate the current global model
4. Train as usually

<div>
<img src="./figs/lightning_client_api.png" alt="Drawing" style="width: 600px;"/>
</div>

You can directly use all the PEFT methods implemented in the NeMo script, by changing the value of [peft_scheme](./nemo_nvflare/megatron_gpt_peft_tuning_config.yaml) in the client configuration shown below accordingly:
* p-tuning
* adapter + p-tuning
* adapter
* LoRa
* ia3

<div>
<img src="./figs/peft_config.png" alt="PEFT config" style="width: 700px;"/>
</div>

In this example, we will use LoRA to run the following experiments.

#### 1. Local training
First, we create the job configuration using the Job API.
Note, the `app_config` options are specific to the app script (`megatron_gpt_peft_tuning.py`) and modify variables in the NeMo config file (`megatron_gpt_peft_tuning_config.yaml`) directly on execution.

At this point, we also modify the local number of clients, local steps, and FL rounds to simulate local training. The PEFT method is [LoRA](https://arxiv.org/abs/2106.09685).

Let's create the job and configure it for simulating local training. To do this, we only run 1 round of FL, with each client running 1000 steps on their local dataset.

In [6]:
import os
from nemo_nvflare.peft_model import PEFTmodel

from nvflare.app_common.workflows.fedavg import FedAvg
from nvflare.app_opt.pt.job_config.base_fed_job import BaseFedJob
from nvflare.job_config.script_runner import ScriptRunner

peft_scheme="lora" # can be either ptuning, adapter, lora, or ia3

# Common configs
peft_scheme_arg=f"model.peft.peft_scheme={peft_scheme}" 
app_script="nemo_nvflare/megatron_gpt_peft_tuning.py"
restore_from_path=f"{os. getcwd()}/megatron_gpt_345m.nemo"
val_files=f"model.data.validation_ds.file_names=[{os. getcwd()}/data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl]"
train_files_prefix=f"model.data.train_ds.file_names=[{os. getcwd()}/data/FinancialPhraseBank-v1.0_split/alpha{alpha}_site"

# Simulate local training on the clients
n_clients = 3
num_rounds=1
trainer_config="trainer.max_steps=1000 trainer.val_check_interval=100"

# Create BaseFedJob with initial model
job = BaseFedJob(
  name=f"peft_{peft_scheme}_local_345M",
  initial_model=PEFTmodel(restore_from_path=restore_from_path),
)

# Define the controller and send to server
controller = FedAvg(
    num_clients=n_clients,
    num_rounds=num_rounds,
)
job.to_server(controller)

# add NeMo config needed on server
job.to_server("nemo_nvflare/megatron_gpt_peft_tuning_config.yaml")

# Add clients
for i in range(1, n_clients+1):
    client_name = f"site-{i}"
    runner = ScriptRunner(script=app_script, script_args=f"{peft_scheme_arg} model.restore_from_path={restore_from_path} {trainer_config} {val_files} {train_files_prefix}-{i}.jsonl]")
    job.to(runner, client_name)

    # add NeMo config needed on each client
    job.to("nemo_nvflare/megatron_gpt_peft_tuning_config.yaml", client_name)

job.export_job("./jobs")

Next, simulate each client training on their local dataset using the FL simulator.

In [ ]:
# required by NeMo models
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

# we run all clients on the same GPU. If you have several GPUs, you can use, gpu="0,1,2"
job.simulator_run(f"/tmp/nvflare/nemo/peft_{peft_scheme}_local_345M_alpha{alpha}", threads=n_clients, gpu="0")

2025-06-09 20:02:37,220 - IntimeModelSelector - INFO - model selection weights control: {}
2025-06-09 20:02:38,948 - TBAnalyticsReceiver - INFO - Tensorboard records can be found in /tmp/nvflare/nemo/peft_lora_local_345M_alpha10.0/server/simulate_job/tb_events you can view it using `tensorboard --logdir=/tmp/nvflare/nemo/peft_lora_local_345M_alpha10.0/server/simulate_job/tb_events`
2025-06-09 20:02:43,650 - TaskScriptRunner - INFO - start task run() with full path: /tmp/nvflare/nemo/peft_lora_local_345M_alpha10.0/site-1/simulate_job/app_site-1/custom/nemo_nvflare/megatron_gpt_peft_tuning.py
2025-06-09 20:02:43,690 - TaskScriptRunner - INFO - start task run() with full path: /tmp/nvflare/nemo/peft_lora_local_345M_alpha10.0/site-2/simulate_job/app_site-2/custom/nemo_nvflare/megatron_gpt_peft_tuning.py
2025-06-09 20:02:43,723 - TaskScriptRunner - INFO - start task run() with full path: /tmp/nvflare/nemo/peft_lora_local_345M_alpha10.0/site-3/simulate_job/app_site-3/custom/nemo_nvflare/mega

[NeMo W 2025-06-09 20:02:48 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2025-06-09 20:02:50 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/lightning/pytorch/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `lightning.pytorch.plugins.precision.MixedPrecision` instead.
    
[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: pipeline_model_parallel_comm_backend in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: virtual_pipeline_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeM

[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Rank 0 has data parallel group : [0]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Ranks 0 has data parallel rank: 0
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Rank 0 has context parallel group: [0]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] All context parallel group ranks: [[0]]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Ranks 0 has context parallel rank: 0
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Rank 0 has model parallel group: [0]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] All model parallel group ranks: [[0]]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] Rank 0 has tensor model parallel group: [0]
[NeMo I 2025-06-09 20:02:51 nemo_logging:393] All tensor model parallel group ranks: [[0]]
[NeMo

[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: pipeline_model_parallel_comm_backend in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: virtual_pipeline_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: hierarchical_context_parallel_sizes in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2025-06-09 20:02:51 nemo_logging:405] The model: MegatronGPTSFTModel() does not have field.name: expert_mode

[NeMo I 2025-06-09 20:02:54 nemo_logging:393] 
    
    ************** Experiment configuration ***********
[NeMo I 2025-06-09 20:02:54 nemo_logging:393] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 1000
      log_every_n_steps: 10
      val_check_interval: 100
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: false
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: false
      checkpoint_callback_params:
        monitor: validation_${model.data.validation_ds.metric.name}
        save_top_k: 1
        mode: min
        save_nemo_on_train_end

...

[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Time building 0 / 1 mem-mapped files: 0:00:00.179721
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Time building 0 / 1 mem-mapped files: 0:00:00.171552
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Configuring DDP for model parallelism.
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Loading data files
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Loading /workspace/data/FinancialPhraseBank-v1.0_split/alpha10.0_site-1.jsonl
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Time loading 1 mem-mapped files: 0:00:00.000523
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Computing global indices
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Configuring DDP for model parallelism.
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : AdapterName.LORA_KQV_ADAPTER
[NeMo I 2025-06-09 20:03:18 nemo_logging:393] Unfrozen adapter : Adap

INFO: 
  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | model       | GPTModel   | 358 M  | train
1 | val_metric  | ModuleList | 0      | train
2 | test_metric | ModuleList | 0      | train
---------------------------------------------------
3.1 M     Trainable params
354 M     Non-trainable params
358 M     Total params
1,432.068 Total estimated model params size (MB)
421       Modules in train mode
0         Modules in eval mode
INFO: 
  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | model       | GPTModel   | 358 M  | train
1 | val_metric  | ModuleList | 0      | train
2 | test_metric | ModuleList | 0      | train
---------------------------------------------------
3.1 M     Trainable params
354 M     Non-trainable params
358 M     Total params
1,432.068 Total estimated model params size (MB)
421       Modules in train mode
0         Modules in eval mode


[NeMo I 2025-06-09 20:03:20 num_microbatches_calculator:228] setting number of microbatches to constant 1
[NeMo I 2025-06-09 20:03:20 num_microbatches_calculator:228] setting number of microbatches to constant 1
[NeMo I 2025-06-09 20:03:20 num_microbatches_calculator:228] setting number of microbatches to constant 1
[NeMo I 2025-06-09 20:03:25 num_microbatches_calculator:228] setting number of microbatches to constant 32
[NeMo I 2025-06-09 20:03:25 num_microbatches_calculator:228] setting number of microbatches to constant 32
[NeMo I 2025-06-09 20:03:25 num_microbatches_calculator:228] setting number of microbatches to constant 32
[NeMo I 2025-06-09 20:03:26 num_microbatches_calculator:228] setting number of microbatches to constant 1
[NeMo I 2025-06-09 20:03:27 num_microbatches_calculator:228] setting number of microbatches to constant 1
[NeMo I 2025-06-09 20:03:27 num_microbatches_calculator:228] setting number of microbatches to constant 1
[NeMo I 2025-06-09 20:03:31 num_microbatche

#### 2. Federated training
Next, we use the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm to adapt the model in a federated scenario. First, create and modify the configuration files again. 
This time, we increase the number of FL rounds and decrease the number of local steps per round to match the federated scenario. 

Here, each client runs LoRA for one 200 steps before sending their local model updates to the server for aggregation. This is repeated for 5 FL rounds. All the other parameters are the same as above.

In [ ]:
# FedAvg setting
num_rounds=5
trainer_config="trainer.max_steps\=200 trainer.val_check_interval\=100"

# Create BaseFedJob with initial model
job = BaseFedJob(
  name=f"peft_{peft_scheme}_fedavg_345M",
  initial_model=PEFTmodel(restore_from_path=restore_from_path),
)

# Define the controller and send to server
controller = FedAvg(
    num_clients=n_clients,
    num_rounds=num_rounds,
)
job.to_server(controller)

# add NeMo config needed on server
job.to_server("nemo_nvflare/megatron_gpt_peft_tuning_config.yaml")

# Add clients
for i in range(1, n_clients+1):
    client_name = f"site-{i}"
    runner = ScriptRunner(script=app_script, script_args=f"{peft_scheme_arg} model.restore_from_path={restore_from_path} {trainer_config} {val_files} {train_files_prefix}-{i}.jsonl]")
    job.to(runner, client_name)

    # add NeMo config needed on each client
    job.to("nemo_nvflare/megatron_gpt_peft_tuning_config.yaml", client_name)

job.export_job("./jobs")

Next, simulate the federated training using FedAvg. 

In [ ]:
# required by NeMo models
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

# we run all clients on the same GPU. If you have several GPUs, you can use, gpu="0,1,2"
job.simulator_run(f"/tmp/nvflare/nemo/peft_{peft_scheme}_local_345M_alpha{alpha}", threads=n_clients, gpu="0")

You can visualize the training process using TensorBoard by running `tensorboard --logdir /tmp/nvflare/nemo` in a new terminal.

## Results
In this scenario, all clients utilize the same validation set, allowing for a direct comparison between the locally p-tuned and federated global models. As anticipated, the FedAvg-trained models achieve a higher overall mean accuracy than those trained solely on their local datasets for different values of `alpha`. This is because the global model has access to all client datasets and can, consequently, generalize better, especially in settings of higher client data heterogeneity.

Below are some examples of how the training data is distributed among the three clients when using different values of `alpha`. The lines show the mean accuracy of local models during training and shaded areas indicate the 95% confidence interval. 
<div>
<img src="./figs/val_accuracy_alpha1.0.svg" alt="Validation accuracy with alpha=1.0" style="width: 400px;"/>
<img src="./figs/val_accuracy_alpha5.0.svg" alt="Validation accuracy with alpha=5.0" style="width: 400px;"/>
<img src="./figs/val_accuracy_alpha10.0.svg" alt="Validation accuracy with alpha=10.0" style="width: 400px;"/>
</div>

## Inference

We can use `model.generate()` to run inference after adapting the model. 
Let's define some test examples to feed to the tuned model to see its predictions.

In [ ]:
prompt = " sentiment:"
test_examples = [f"The products have a low salt and fat content .{prompt}",
    f"The agreement is valid for four years .{prompt}",
    f"Diluted EPS rose to EUR3 .68 from EUR0 .50 .{prompt}",
    f"The company is well positioned in Brazil and Uruguay .{prompt}",
    f"Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier .{prompt}",
]

First, we need to convert the best global PEFT model into a NeMo ckpt.

In [ ]:
import os
from nemo_nvflare.utils import convert_global_to_ckpt
server_workspace = f"/tmp/nvflare/nemo/peft_{peft_scheme}_fedavg_345M_alpha{alpha}/server/simulate_job/app_server"
global_model_filepath = os.path.join(server_workspace, "best_FL_global_model.pt")
assert global_model_filepath.endswith(".pt")
ckpt_path = global_model_filepath.replace(".pt", ".ckpt")
convert_global_to_ckpt(global_model_filepath, ckpt_path)

Next, we will load the global model.

In [ ]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTSFTModel
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronLMPPTrainerBuilder
from nemo.collections.nlp.parts.peft_config import PEFT_CONFIG_MAP
from omegaconf import OmegaConf

# Load model configuration inference of the global model
cfg = OmegaConf.load("nemo_nvflare/megatron_gpt_peft_fl_eval_config.yaml")

# Build trainer
trainer = MegatronLMPPTrainerBuilder(cfg).create_trainer()

# Set restore from paths with pre-trained model(s)
cfg.model.restore_from_path = "megatron_gpt_345m.nemo"

# Set the global peft weights
cfg.model.peft.restore_from_path = ckpt_path

model_cfg = MegatronGPTSFTModel.merge_cfg_with(cfg.model.restore_from_path, cfg)
model = MegatronGPTSFTModel.restore_from(cfg.model.restore_from_path, model_cfg, trainer=trainer)
peft_cfg_cls = PEFT_CONFIG_MAP[cfg.model.peft.peft_scheme]

print("PEFT Weights will be loaded from", cfg.model.peft.restore_from_path)
model.load_adapters(cfg.model.peft.restore_from_path, peft_cfg_cls(model_cfg))
model.freeze()

print("Model initialized", type(model))

Run the model

In [ ]:
# Adjust the sampling parameters as needed
sampling_params = {
    "use_greedy": True,
    "temperature": 1.0,
    "top_k": 0,
    "top_p": 0.9,
    "repetition_penalty": 1.2,
    "add_BOS": False,
    "all_probs": False,
    "compute_logprob": False,
    "end_strings": ["<|endoftext|>", "<extra_id_1>"],
}

response = model.generate(inputs=test_examples, length_params=None, sampling_params=sampling_params)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print("-" * 30)
    print(result)

The expected output of a well-trained model looks something like this. Note, the test sentences do not include ground truth labels.

>      The products have a low salt and fat content . sentiment: neutral
>      ------------------------------
>      The agreement is valid for four years . sentiment: neutral
>      ------------------------------
>      Diluted EPS rose to EUR3 .68 from EUR0 .50 . sentiment: positive
>      ------------------------------
>      The company is well positioned in Brazil and Uruguay . sentiment: positive
>      ------------------------------
>      Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier . sentiment: negative
>      ------------------------------